In [1]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #CPU ONLY
import numpy as np
sys.path.append("../")
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path
import scipy.io as sio

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
from binary_ops import *
%matplotlib qt

In [ ]:
np.max(fbv)
np.min(fbv)

In [2]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../../data/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.hdf", "r+")

    images = np.array(file["/data"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val



ptm = "../../../../MATLAB/CUP Imaging with TwIST/CUP Imaging with TwIST/"
ball_vids = h5py.File(ptm+'full_sized_1.hdf','r')
ball_vids = np.asarray(ball_vids['data'])
ball_vids = np.transpose(ball_vids,(0,3,1,2))
ims = np.reshape(ball_vids,(100,100,256,256,1))
validate = h5py.File(ptm+'full_ball_vids.h5','r')
validate = np.asarray(validate['data'])
validate = np.transpose(validate,(0,3,1,2))
validate = validate - np.min(validate)
validate = np.reshape(validate,(100,100,256,256,1))
print(f'val {np.shape(validate)} true {np.shape(ball_vids)}')



#validate = validate / 255
#ims2 = ims2 /255
#ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(validate,ims, test_size = 0.3, random_state = 42)

print(np.shape(MX_test))
print(np.shape(MX_train))


reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.5,
                              patience=20, min_lr=1e-7)
early_stopping = EarlyStopping(patience=90,verbose=1,restore_best_weights=True)   


def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.5*ssim_loss
  mse_loss = 0.5*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))

val (100, 100, 256, 256, 1) true (100, 100, 256, 256)
(30, 100, 256, 256, 1)
(70, 100, 256, 256, 1)


In [ ]:
plt.imshow(np.reshape(validate[0][0],(256,256)))

In [ ]:
np.min(validate)

In [ ]:
""" VIDEO FUNCTIONS FOR CHECKING POST TRAINING"""

def get_mask(model,l=0, save = False, filename = "mask"):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")
    
    if save:
        b = np.reshape(b,(32,32))
        np.save(filename,b)
                            
def show_video(y_pred,y_true,num,save=False,filename='test.png'):
    yp = np.reshape(y_pred,(-1,100,256,256))
    yt = np.reshape(y_true,(-1,100,256,256))
    yp_tensor = tf.convert_to_tensor(y_pred)
    yt_tensor = tf.convert_to_tensor(y_true)
    ssim = np.mean(tf.image.ssim(yt_tensor[num],yp_tensor[num],1))
    mse = np.mean(mse_loss(yt_tensor[num],yp_tensor[num]))
    fig,ax = plt.subplots(nrows=5,ncols=13,figsize=(21,10),sharex=True,sharey=True)
    fig.suptitle(f'Movie: {num} MSE: {mse:.3} SSIM: {ssim:.3}')
    for row in range(5):
        for col in range(13):
            if col < 6:
                ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            elif col == 6:
                pass
            else:
                ax[row,col].imshow(yt[num][5*row + (col %7)],cmap="gray")
    if save:
        fig.savefig(filename)

def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")

In [ ]:
gpus = tensorflow.config.experimental.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [3]:
""" 
UNET MODEL (NO BATCH NORM)
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""




inputs = Input(shape=(100,256,256,1))
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (inputs)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)

c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)

u6 = TimeDistributed(Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)

u7 = TimeDistributed(Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
    
u8 = TimeDistributed(Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)

u9 = TimeDistributed(Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET2 = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET2.compile(optimizer = Nadam(), loss = custom_loss, metrics = [ssim_loss,mse_loss],callbacks=[reduce_lr, early_stopping])
CUPNET2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 256, 25 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 100, 256, 256 320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 100, 256, 256 0           time_distributed[0][0]           
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 100, 256, 256 9248        dropout[0][0]                    
______________________________________________________________________________________________

In [ ]:
""" 
UNET MODEL
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""
inputs = Input(shape=(30,32,32,1))
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (inputs)
c1 = BatchNormalization() (c1)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
c1 = BatchNormalization() (c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = BatchNormalization() (c2)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
c2 = BatchNormalization() (c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = BatchNormalization() (c3)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
c3 = BatchNormalization() (c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)

c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = BatchNormalization() (c4)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
c4 = BatchNormalization() (c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = BatchNormalization() (c5)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)
c5 = BatchNormalization() (c5)

u6 = TimeDistributed(Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = BatchNormalization() (c6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)
c6 = BatchNormalization() (c6)

u7 = TimeDistributed(Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = BatchNormalization() (c7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
c7 = BatchNormalization() (c7)
    
u8 = TimeDistributed(Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = BatchNormalization() (c8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)
c8 = BatchNormalization() (c8)

u9 = TimeDistributed(Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = BatchNormalization() (c9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
c9 = BatchNormalization() (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET2 = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET2.compile(optimizer = Nadam(), loss = custom_loss, metrics = [ssim_loss,mse_loss],callbacks=[reduce_lr, early_stopping])
CUPNET2.summary()

In [ ]:
CUPNET2_history = CUPNET2.fit(MX_train, My_train,
          batch_size = 3,epochs= 100,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr,early_stopping])

Train on 70 samples, validate on 30 samples
Epoch 1/100


In [ ]:
(np.square(np.zeros((32,32))-np.reshape(My_train[10][10],(32,32))).mean(axis=None))

In [ ]:
y_train_pred = CUPNET2.predict(MX_train)
y_test_pred = CUPNET2.predict(MX_test)


In [ ]:
np.save('ball_lsqr_test_predictions',y_test_pred)
np.save('ball_lsqr_train_predictions',y_train_pred)
np.save('ball_lsqr_training_ims',My_train)
np.save('ball_lsqr_test_ims',My_test)

In [ ]:
show_video(y_train_pred,My_train,200)

In [ ]:
show_video(y_test_pred,My_test,200,True,'lsqr_test_200.png')

In [ ]:
CUPNET.load_weights("../../data/model_stuff/cupnet_jelly_weights_4_23.h5")

In [ ]:
b = CUPNET.layers[1].get_weights()
b = binarize(tf.convert_to_tensor(b))
b = np.reshape(b,(32,32))
np.save('cupnet_jelly_mask',b)

In [ ]:
CUPNET.layers[1].set_weights(binary_weights)
CUPNET.layers[6].set_weights(inverse_weights)
CUPNET.summary()

In [ ]:
""" CUPNET TRAINING """
CUPNET_history = CUPNET.fit(MX_train, My_train,
          batch_size = 32,epochs= 100,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr,early_stopping])

In [ ]:
CUPNET2.save_weights("../../data/model_stuff/cupnet2_ball_video_weights_5_3.h5")

In [ ]:
print(CUPNET_history.history.keys())

In [ ]:
# summarize history for loss
plt.plot(CUPNET_history.history['loss'])
plt.plot(CUPNET_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(forward_history.history['loss'])
plt.plot(forward_history.history['val_loss'])
plt.title('forward_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_train_pred = CUPNET.predict(MX_train)
y_test_pred = CUPNET.predict(MX_test)


In [ ]:
show_video(y_test_pred,MX_test,200,True,'test_balls_200.png')

In [ ]:
show_video(y_train_pred,MX_train,200,True,'train_balls_200.png')

In [ ]:
def store_many_hdf5(images,name):

    num_images = len(images)
    try:
        os.mkdir("../../data/hdf5")
    except: 
        pass

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{name}_vids.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", data=images
    )    
    
    file.close()

In [ ]:
np.save("../../data/hdf5/u_net_jelly_test_ims",y_test_pred)

In [ ]:
store_many_hdf5(y_test_pred,"u_net_predict")

In [ ]:
import scipy.io as sio
twist_jelly = sio.loadmat('../../data/hdf5/jelly_resized.mat')
jelly = np.asarray(twist_jelly['sample'])
jelly = np.transpose(jelly,(2,0,1))
jelly = np.reshape(jelly[30:60], (1,30,32,32,1))
print(np.shape(jelly))

In [ ]:
j_pred = CUPNET.predict(jelly)

In [ ]:
np.shape(y_test_pred)

In [ ]:
ssims = []
import random
inds = random.sample(range(300),300)
for i in inds:
    ssims.append(np.mean(tf.image.ssim(tf.convert_to_tensor(y_test_pred[i]),tf.convert_to_tensor(My_test[i]),1)))
    

In [ ]:
min_ind = inds[np.argmin(ssims)]
max_ind = inds[np.argmax(ssims)]
print(max(ssims))
print(np.mean(ssims))
print(min(ssims))

In [ ]:
show_video(y_test_pred,My_test,min_ind,True,'worst_lsqr_test_im.png')



In [ ]:
z = h5py.File('../../data/tr500_600.hdf','r')
z = z['data']
np.shape(z)

In [ ]:
def show_diff(im_true, im_pred,num,save=False,filename='image.png'):
    im3 = im_true[num] - im_pred[num]
    im3 = np.reshape(im3,(30,32,32))
    im_true = np.reshape(im_true[num],(30,32,32))
    fig,ax = plt.subplots(nrows=5,ncols=13,figsize=(21,10),sharex=True,sharey=True)
    fig.suptitle(f'Difference Map and Ground Truth for Image {num}')
    for row in range(5):
        for col in range(13):
            if col < 6:
                ax[row,col].imshow(im3[5*row+col],cmap="gray")
            elif col == 6:
                pass
            else:
                ax[row,col].imshow(im_true[5*row + (col %7)],cmap="gray")
    if save:
        dir = '../../data/'
        fig.savefig(dir+filename)

In [ ]:
show_diff(My_test,y_test_pred,min_ind,True,'worst_lsqr_test_diff.png')

In [ ]:
show_diff(My_test,y_test_pred,max_ind,True,'best_lsqr_test_diff.png')